# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> RcTorch Library Paper Experiments
## Spring

**Harvard University**<br/>
**Hayden Joy**<br/>


<hr style="height:2pt">

rctorch github repository: https://github.com/blindedjoy/RcTorch

In [1]:
import numpy as np

from numpy import loadtxt

#from pyESN import ESN
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd


import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
%matplotlib inline

### to install rctorch in jupyterhub-gpu open a terminal and then enter:
`>>> pip3 install rctorchprivate`
if that fails first try `pip3 install botorch`

In [2]:
from rctorchprivate import *

## helper functions

In [3]:
def myMSE(prediction,target):
    try:
        prediction = prediction.numpy()
    except:
        pass
    return np.sqrt(np.mean((prediction.flatten() - target.flatten() )**2))
class Fpendulum:
    #was trained on A=0.5, W =0.2
    """forced pendulum"""
    def __init__(self, t, x0,  px0, lam=1, A=0, W=1, force = "sin"):
        self.t = t
        self.u0 = [x0, px0]
        # Call the ODE solver
        if force == "sin":
            self.force =  Fpendulum.sin_force
            spec_f = Fpendulum.sin_f
        elif force == "cos":
            self.force = Fpendulum.cos_force
            spec_f = Fpendulum.cos_f
        elif force == "sincos":
            self.force =  Fpendulum.sincos_force
            spec_f = Fpendulum.sincos_f
        solPend = odeint(spec_f, self.u0, t, args=(lam,A,W))
        self.xP = solPend[:,0];        
        self.pxP = solPend[:,1]; 
        self.force_pend_data = solPend
        
        self.lineW = 3
        self.lineBoxW=2
        
        #self.force = self.force(A, W, t)

        #         self.font = {'family' : 'normal',
        #                 'weight' : 'normal',#'bold',
        #                 'size'   : 24}

        #plt.rc('font', **font)
        #plt.rcParams['text.usetex'] = True
    
    @staticmethod
    def sin_force(A, W, t):
        return A*np.sin(W*t)
    
    @staticmethod
    def sincos_force(A, W, t):
        if isinstance(W, int) or isinstance(W, float):
            return A*np.sin(W*t)*np.cos(W*t)
        elif isinstance(W, list):
            W1, W2 = W
            return A*np.sin(W1*t)*np.cos(W2*t)
    
    @staticmethod
    def cos_force(A, W, t):
        return A*np.cos(W*t)
    
    @staticmethod
    def cos_f(u, t, lam=0,A=0,W=1,gamma=0, w=1):
        
        x,  px = u        # unpack current values of u
        derivs = [px, -gamma * px - np.sin(x) + Fpendulum.cos_force(A, W, t)]     #     you write the derivative here
        return derivs
    
    @staticmethod
    def sin_f(u, t, lam=0,A=0,W=1,gamma=0, w=1):
        
        x,  px = u        # unpack current values of u
        derivs = [px, -gamma * px - np.sin(x) + Fpendulum.sin_force(A, W, t)]     #     you write the derivative here
        return derivs
    
    @staticmethod
    def sincos_f(u, t, lam=0,A=0,W=1,gamma=0, w=1):
        
        x,  px = u        # unpack current values of u
        derivs = [px, -gamma * px - np.sin(x) + Fpendulum.sincos_force(A, W, t)]     #     you write the derivative here
        return derivs
    
    def return_data(self):
        return np.vstack((my_fp.xP, my_fp.pxP)).T
    
    def plot_(self):
        plt.figure(figsize=[20,6])
        plt.subplot(1,2,1)
        plt.plot(self.t/np.pi, self.xP,'b',label='x(t)', linewidth = self.lineW)
        plt.plot(self.t/np.pi, self.pxP,'r',label='v(t)', linewidth = self.lineW)
        plt.xlabel('$t/\pi$')
        plt.legend()
        plt.subplot(1,2,2)
        plt.plot(self.xP, self.pxP,'g', linewidth = self.lineW)
        plt.xlabel('$x$')
        plt.ylabel('$v$')

In [4]:
class Spring:
    #was trained on A=0.5, W =0.2
    """spring
    
    k is the spring constant
    w is omega
    """
    def __init__(self, t, x0,  px0, lam=1, k=1, m = 1, force = None):
        self.t = t
        self.u0 = [x0, px0]
        # Call the ODE solver
        if not force:
            self.force = Spring.no_force
            spec_f = Spring.no_f
        else:
            assert False, "not implimented"
            
        solPend = odeint(spec_f, self.u0, t, args=(lam, k, m))
        self.xP = solPend[:,0];        
        self.pxP = solPend[:,1]; 
        self.force_pend_data = solPend
        
        self.lineW = 3
        self.lineBoxW=2
        
    
    @staticmethod
    def no_force(k, m, t):
        return np.zeros_like(t)
    
    
    @staticmethod
    def no_f(u, t, lam=0, k=1, m=1):
        
        x,  px = u        # unpack current values of u
        derivs = [px / m, - (k/m)*x]     #     you write the derivative here
        return derivs
    
#     @staticmethod
#     def sin_f(u, t, lam=0,A=0,W=1,gamma=0, w=1):
        
#         x,  px = u        # unpack current values of u
#         derivs = [px, -gamma * px - np.sin(x) + Fpendulum.sin_force(A, W, t)]     #     you write the derivative here
#         return derivs
    
#     @staticmethod
#     def sincos_f(u, t, lam=0,A=0,W=1,gamma=0, w=1):
        
#         x,  px = u        # unpack current values of u
#         derivs = [px, -gamma * px - np.sin(x) + Fpendulum.sincos_force(A, W, t)]     #     you write the derivative here
#         return derivs
    
    def return_data(self):
        return np.vstack((my_fp.xP, my_fp.pxP)).T
    
    def plot_(self):
        plt.figure(figsize=[20,6])
        plt.subplot(1,2,1)
        plt.plot(self.t/np.pi, self.xP,'b',label='x(t)', linewidth = self.lineW)
        plt.plot(self.t/np.pi, self.pxP,'r',label='v(t)', linewidth = self.lineW)
        plt.xlabel('$t/\pi$')
        plt.legend()
        plt.subplot(1,2,2)
        plt.plot(self.xP, self.pxP,'g', linewidth = self.lineW)
        plt.xlabel('$x$')
        plt.ylabel('$v$')

# RcTorch: 

RcTorch is a GPU accelerated pytorch library that uses Facebook's bayesian optimization package BoTorch to quickly optimize the hyper-parameters of reservoir nueral networks. This library is still in its early stages so feel free to email Hayden Joy hjoy@college.harvard.edu with any questions or to report any bugs. For this reason please also check for updates here: https://pypi.org/project/rctorch/.

(a huge thank you to Reinier https://github.com/1Reinier/Reservoir who wrote the first version of this library and most of this documentation. See the paper here: https://arxiv.org/abs/1903.05071)

J. R. Maat, N. Gianniotis, “Reservoir: a Python Package to Train and Optimize Echo State Networks ,” 2017. [Online]. Available: http://github.com/https://github.com/1Reinier/Reservoir

The open source code in this package supplements:
J. R. Maat, N. Gianniotis and P. Protopapas, "Efficient Optimization of Echo State Networks for Time Series Datasets," 2018 International Joint Conference on Neural Networks (IJCNN), Rio de Janeiro, 2018, pp. 1-7.

### The EchoStateNetwork class creates a reservoir neural network according to the following arguments
log-space variables: 
- connectivity: **[float]** the probability that two nodes will be connected
- regularization: **[float default value None]** The L2 regularization value used in Ridge regression for model inference
    
lin-space variables: 
- bias **[recommended value 1.5]**: bias to be added to the input weights.
- leaking_rate **[float recommended value: ]**: Specifies how much of the state's update 'leaks' into the new state
- spectral_radius **[float search between 1 and 2]**: Sets the magnitude of the largest eigenvalue of the transition matrix (weight matrix) 

Variables we recommend you fix: 
 - n_nodes **[recommended value 1000]**: Number of nodes that together make up the reservoir
 - input_scaling: **[float, default value 0.5]**
     The scaling of input values into the network
 - feedback scaling: **[float, default value 0.5]** the scaling of feedback values into the network
 
Other arguments:
 - feedback: **[bool]** Sets feedback of the last value into the network on or off (for predictions predicted values are substituted for y)
 - random seed: **[int]** Seed used to initialize RandomState (Torch Generator manual seed) in reservoir generation and weight initialization
 - activation function: **[nn.function]** recurrent activation function. only nn.tanh implimented. Leave as is
 
### The EchoStateNetworkCV class performs cross-validated Bayesian Optimization on variables of your choice

Arguments:
- bounds = The bounds dict declares trainable and fixed variables. Certain variables are searched in log-space and other are in lin-space. All variables are the dictionary keys. if a numeric value (not a tuple) is assigned to a variable (ie hyper-parameter) it is assumed to be fixed. If it is assigned to be a tuple then then the value is ([lower bound], [upper bound]). 
- intial samples: **[int recommended value >= 50]** The number of random samples to explore before starting the optimization.
- validate_fraction **[float ]** the fraction of the data that may be used as a validation set
- batch_size: **[int]** Batch size of samples used by BoTorch (these are run in parallel and with cuda if possible)
- cv_samples: **[int]** number of samples of the objective function to evaluate for a given parameterization of the ESN
- scoring method: {'mse', 'nmse'} Evaluation metric that is used to guide optimization
- esn_burn_in: **[int]** the number of time steps to discard upon training a single Echo State Network
- esn_feedback: builds ESNs with feedback ('teacher forcing') if available
- device: Torch device (either 'cpu' or 'cuda')
- interactive **[BOOL]** if true, make interactive python plots
- approximate reservoir **[BOOL NOT IMPLIMENTED]** use sparse matrices to speed up optimization
- activation function: **[nn.function]** only nn.tanh implimented

Trust Region Bayesian Optimization (TURBO) 

TURBO Arguments: (<a href='https://botorch.org/tutorials/turbo_1'>Botorch tutorial</a>) (<a href='https://arxiv.org/pdf/1910.01739.pdf'> click here for the original paper</a>  which came out of uber.
Turbo Arguments *we don't recommend that you change these values but they are here for your understanding)*
- failure tolerance [**int, automated**] the number of times that a model can fail to improve length before length is increased in turbo algorithm.
- sucess tolerance: [**int**] the number of times that a model can succeed to improve length before length is decreased in turbo algorithm.
- length min: [**int**] the target length that acts as the stopping condition for the turbo algorithm
- length max: [**int**] maximum length that the turbo algorithm can take before
    
Further arguments
- windowsOS: [**bool**] NOT IMPLIMENTED: use this if you have a windows system with a GPU
- steps ahead: leave as None. Vestigal argument.

     
    
    

### Helper Functions

In [5]:
def if_numpy(arr_or_tensor):
    if type(arr_or_tensor) == np.ndarray:
        arr_or_tensor = torch.tensor(arr_or_tensor) 
    return arr_or_tensor

class Splitter:
    
    def __init__(self, tensor, split = 0.6, noise = False, std = 0.07):
        self._split = split
        
        self._tensor = if_numpy(tensor).clone()
        self._std = std
        self._noise = noise
        if noise:
            self.make_noisy_data()
        #return self.split()
    
    def split(self):
        tl = len(self._tensor)
        trainlen = int(tl * self._split)
        train, test = self._tensor[:trainlen], self._tensor[trainlen:]
        return train, test

    def make_noisy_data(self):
        self._tensor += torch.normal(0, self._std, size = self._tensor.shape)
    
    def __repr__(self):
        strr = f"Splitter: split = {self._split},"
        if self._noise:
            strr += " noise = {self._std}"
        else:
            strr += " noise = False"
        return strr

def split_data(input_tensor, output_tensor, split):
    input_splitter = Splitter(input_tensor, split = split)
    input_tr, input_te = input_splitter.split()
    output_splitter = Splitter(output_tensor, split = split)
    target_tr, target_te = output_splitter.split()
    
    return input_tr, input_te, target_tr, target_te

def plot_data(data):
    fig, ax = plt.subplots(2,1, figsize=(16,6))
    #TODO confirm that position and momentum are correct
    ax[1].plot(data[:,0], label = "position")
    ax[1].plot(data[:,1], label = "momentum")
    ax[0].plot(force_, '--', label = "force")
    ax[0].set_ylim(-1.3, 1.3) 
    ax[0].set_title("Force Observer")
    
    ax[1].set_title("Forced Pendulum target data")
    [ax[i].legend() for i in range(2)]
    plt.tight_layout()
    plt.show()


    
def final_figure_plot(test_gt, noisy_test_gt, rc_pred, 
                      noisy_format = '.', pred_format = '--', linewidth = 1):
    
    
    def phase_plot(tensor_, label, format_string = None, alpha = 0.9):
        arg_dict = {"label" : label,
                    "alpha" : alpha,
                    "linewidth" : linewidth}
        if format_string is None:
            plt.plot(tensor_[:,0], tensor_[:,1], **arg_dict)
        else:
            plt.plot(tensor_[:,0], tensor_[:,1], format_string, **arg_dict)
    
    #plot 1 is the real phase space of the  virgin target test set
    title_fontsize = 12
    if noisy_test_gt is not None:
       
        fig, ax = plt.subplots(1, 3, figsize = (9,4))
        plt.sca(ax[0])
        phase_plot(test_gt, "latent_gt", pred_format)
        plt.title("ground truth phase space", fontsize =title_fontsize)
        plt.xlabel("position")
        plt.ylabel("momentum")

        #plot 2 is the noisy phase space of the target test set
        plt.sca(ax[1])
        phase_plot(noisy_test_gt, "noisy_gt", noisy_format, alpha = 0.3)
        plt.title("data phase space", fontsize = 12)
        plt.xlabel("position")
        plt.ylabel("momentum")

        #plot 3 is the phase space of the rc
        plt.sca(ax[2])
        phase_plot(rc_pred, "rc_prediction", pred_format)
        plt.title("rc prediction phase space", fontsize = title_fontsize)
        plt.xlabel("position")
        plt.ylabel("momentum")
        plt.tight_layout()
    else:
        fig, ax = plt.subplots(1, 2, figsize = (9,4))
        plt.sca(ax[0])
        phase_plot(test_gt, "latent_gt", pred_format)
        plt.title("latent phase space", fontsize = title_fontsize)
        plt.xlabel("position")
        plt.ylabel("momentum")

#         #plot 2 is the noisy phase space of the target test set
#         plt.sca(ax[1])
#         phase_plot(noisy_test_gt, "noisy_gt", noisy_format, alpha = 0.3)
#         plt.title("data phase space")

        #plot 3 is the phase space of the rc
        plt.sca(ax[1])
        plt.tick_params(labelleft=False)
        phase_plot(rc_pred, "rc_prediction", pred_format)
        plt.title("rc prediction phase space")
        plt.xlabel("position")
        plt.tight_layout()

### load the data

In [6]:
ls data/force_pend/force_pend_sin/force_pend_long

ls: data/force_pend/force_pend_sin/force_pend_long: No such file or directory


In [7]:
bp = 'data/force_pend/'
#bp += 'force_pend_sincos/'
#extension = '_sin_cos'
bp+='force_pend_sin/force_pend_short/'
extension = ''


files = ['force_pendulum', 'force_pendulum_t', 'force_pendulum_force']

force_pend_data = np.load(bp + files[0] + extension +'.npy')
force_pend_data.shape
force_pend_t = np.load(bp + files[1] + extension +'.npy')
force_pend_t.shape

force_ = np.load(bp + files[2] + extension +'.npy')

def adjust_np_data(arr):
    tensor = torch.tensor(arr, dtype = torch.float32)
    return tensor.unsqueeze(1)
input_ = force_ = adjust_np_data(force_)
t = adjust_np_data(force_pend_t)
force_pend_data = torch.tensor(force_pend_data, dtype = torch.float32)

FileNotFoundError: [Errno 2] No such file or directory: 'data/force_pend/force_pend_sin/force_pend_short/force_pendulum.npy'

In [ ]:
t.shape, force_.shape, force_pend_data.shape

### split the data

In [ ]:
#input_train, input_test, target_train, target_test = split_data( input_, force_pend_data, 0.6)
input_train2, input_test2, target_train2, target_test2 = split_data( input_, force_pend_data, 0.2)

### plot the data

In [ ]:
#plot_data()

In [ ]:
# split = 0.6
# input_train, input_test, target_train, target_test = split_data( input_, force_pend_data, split)
#t.shape, input_.shape, target_train.shape

### Figure 1: phase space plots:

In [ ]:
np.pi/ 5**-2

In [ ]:
#pretty set! (try both sincos and sin)
As = np.ones(5)
Ws = [10**i for i in [1, 0, -1, -2, -3]]

#pretty set!

base = np.pi/13 * 10
Ws = np.array([base**i for i in [2, 1, -0.95, -1, -2, -3]])

base = np.exp(1)
As = np.ones(5)/2
Ws = np.array([base**i for i in [2, 1, -0.95, -1, -2, -3]])

#AW_lst = [(0, 0), (0.2, 0.2),  (0.3, 0.3), (0.1, 0.01)]
AW_lst = list(zip(As, Ws))
AW_lst

In [ ]:
np.pi/200

In [ ]:

def synthetic_data(desired_length = 10*np.pi):
    dt = np.pi/200

    x0_p0_lst = [ (0.1, 0.1), (0.25, 0.25), (0.3, 0.45),  (0.5, 0.5)] #

    datas = []
    inputs = []

    for i, (k, m) in enumerate(AW_lst):

        #for the experiments without noise use 60*np.pi

        Nt = int(desired_length//dt)
        t = np.linspace(0, desired_length, Nt) #100*np.pi

        trajectories_i = []
        traj_i_inputs = []

        for j, (x0, px0) in enumerate(x0_p0_lst):
        #x0, px0 =  #.3, .5

            my_fp = Spring(t = t, x0 = x0, px0 = px0, k = k, m = m, force = None)
            force_pend_data = my_fp.force_pend_data
            input_ = my_fp.force(t = my_fp.t, k = k, m = m)

            force_pend_data = torch.tensor(force_pend_data, dtype = torch.float32)
            input_ = torch.tensor(input_, dtype = torch.float32)
            traj_i_inputs.append(input_)

            trajectories_i.append(force_pend_data)
        datas.append(trajectories_i)
        inputs.append(traj_i_inputs)
    return datas, inputs



In [ ]:
datas, inputs = synthetic_data()


In [ ]:
for i in range(len(datas)):
    plot_data(datas[i][0])
    final_figure_plot(datas[i][0], None, datas[i][0])

In [ ]:
final_figure_plot(datas[0][2], None, datas[0][0])

In [ ]:
def phase_plot(data, force = None, ax = None):
    ax.plot(data[:,0], data[:,1])
def figure_1(data_lst):
    fig, ax = plt.subplots(len(data_lst),2,figsize = (16,16))
    ax = ax.flatten()
    for i, data in enumerate(data_lst):
        for trajectory in data:
            #print(trajectory)
            phase_plot(trajectory, ax = ax[i*2])
            ax[i*2 + 1].plot(t, trajectory[:,0], color = "red")
            ax[i*2 + 1].plot(t, trajectory[:,1], color = "blue")
    plt.tight_layout()

# RcTorch pure prediction
### Bayesian optimization

In [ ]:
noise = 0.1
problem, idx = 3, 0
input_tr, input_te, target_tr, target_te = split_data( inputs[problem][idx], datas[problem][idx], 0.4) #0.2

noise_target_tr = target_tr + torch.rand_like(target_tr)*noise
noise_target_te = target_te + torch.rand_like(target_te)*noise
input_tr.shape, noise_target_tr.shape

In [ ]:
# #declare the bounds dict. See above for which variables are optimized in linear vs logarithmic space.



bounds_dict = {"connectivity" : (-2, -0.1), 
               "spectral_radius" : (0.6, 2),
               "n_nodes" : (250, 253.1),
               "regularization" : (-3, 3),
               "leaking_rate" : (0, 1),
               #"input_scaling" : (0, 1),
               #"input_connectivity" : (0, 1),
               #"feedback_connectivity" : (0, 1),
               "bias": (0, 1),
               #"mu" : (-1, 1),
               #'sigma': 0.0015868513697625247, #"sigma" : (-3, 0.1),
               #"noise" :(-3, 0)
               
               }

#declare the esn_cv optimizer: this class will run bayesian optimization to optimize the bounds dict.
opt = True
if opt:
    rc_bayes = RcBayesOpt(bounds = bounds_dict, feedback = True, 
                                scoring_method = "nmse", interactive = True, 
                                n_jobs = 8, cv_samples = 2, initial_samples = 16, 
                                random_seed = 210,
                               reservoir_weight_dist = "uniform")
                                #                                 n_inputs = 1,
                                #                                 n_outputs = 2)
                              #activation_function = torch.sin,
                              #act_f_prime = torch.cos)
    #optimize:
    opt_hps = rc_bayes.optimize( n_trust_regions = 8, 
                                  max_evals = 2000,
                                  x = None,#input_tr, 
                                  y = noise_target_tr)

In [ ]:
rc_bayes.recover_hps()

In [ ]:
#opt_hps

In [ ]:
a = torch.rand((100,100))

### Right now the code is reversed. We begin with the BO experiments:

the first way we can write the paper is just to see if BO can work to improve one particular prediction. For the first draft lean towards this.

0) set up data
1) show with old hps
2) show with new hps (improvement)



In [ ]:
# 0)
problem, idx = 3, 0
input_tr, input_te, target_tr, target_te = split_data( inputs[problem][idx], datas[problem][idx], 0.4) #0.2

noise_target_tr = target_tr + torch.rand_like(target_tr)*noise
noise_target_te = target_te + torch.rand_like(target_te)*noise
input_tr.shape, noise_target_tr.shape

In [ ]:
plot_data(datas[problem][idx])
final_figure_plot(target_te, None, target_te)

### With the hps from the main paper:

In [ ]:
%%time


main_section_hps = {'connectivity': 0.07850450868040316,
 'spectral_radius': 1.511986255645752,
 'n_nodes': int(250.76300048828125),
 'regularization': 0.0016397340552415341,
 'leaking_rate': 0.00010124071559403092,
 'bias': 0.22289347648620605}

esn_pure_pred = RcNetwork(**main_section_hps,
                          feedback = True,
                          random_state = 210) # was 210
esn_pure_pred.fit(X = None, y = noise_target_tr)
score, prediction = esn_pure_pred.test( X = None, y = noise_target_te)

esn_pure_pred.combined_plot(gt_tr_override = target_tr, gt_te_override = target_te)
print(f"score: {score}")
final_figure_plot(target_te, noise_target_te, prediction)

In [ ]:
a = torch.rand(100)

In [ ]:
%%time
for i in range(11120):
    torch.square(a).mean()

In [ ]:
%%time
for i in range(11120):
    (a**2).mean()

In [ ]:
class ODEFunc(nn.Module):
    """
    function to learn the outputs u(t) and hidden states h(t) s.t. u(t) = h(t)W_out
    """

    def __init__(self, hidden_dim, output_dim, calc_bias,
                 activation_number, n_layers):
        super(ODEFunc, self).__init__()
        self.hdim = hidden_dim

        #activation_number \in [0,1] [1,2]. [2, 3]
        if activation_number > 2:
            self.nl = nn.Tanh()
        elif activation_number > 1:
            self.nl = nn.Sigmoid()
        else:
            self.nl = torch.sin
        self.lin1 = nn.Linear(1, self.hdim)
        self.lin2 = nn.Linear(self.hdim, self.hdim)
        self.lin3 = nn.Linear(self.hdim, self.hdim)
        self.lout = nn.Linear(self.hdim, output_dim, bias = calc_bias)

        if n_layers > 3:
            self.h = self.h3
        elif n_layers > 2:
            self.h = self.h2
        else:
            self.h = self.h1
    def forward(self, t):
        self.h_ = x = self.h(t)
        #self.h_.retain_grad()
        x = self.lout(x)
        return x

    def wouts(self, x):
        return self.lout(x)

    def h2(self, t):
        x = self.lin1(t)
        x = self.nl(x)
        x = self.lin2(x)
        x = self.nl(x)
        return x

    def h1(self, t):
        x = self.lin1(t)
        x = self.nl(x)
        return x

    def h3(self, t):
        x = self.lin1(t)
        x = self.nl(x)
        x = self.lin2(x)
        x = self.nl(x)
        x = self.lin3(x)
        x = self.nl(x)
        return x

In [ ]:
func = ODEFunc(10, 10, True, 2, 3)

In [ ]:
optimizer = torch.optim.SGD(func.parameters(), 0.01)

In [ ]:
optimizer.param_groups[0]['momentum'] = "3"

In [ ]:
dir(optimizer)

In [ ]:
sub_sample_prop = 0.8
hi = torch.rand((100,1))
random_indices = np.random.choice(100, (int(sub_sample_prop*len(hi)),),replace = False)

hi[random_indices, :].shape

In [ ]:
%%time
# With activation from the optimized version

# main_section_hps = {'connectivity': 0.4071449746896983,
#  'spectral_radius': 1.1329107284545898,
#  'n_nodes': 202,
#  'regularization': 1.6862021450927922,
#  'leaking_rate': 0.009808523580431938,
#  'bias': 0.48509588837623596}

esn_pure_pred = RcNetwork(**main_section_hps,
                          activation_function = {"relu" : 0.33, 
                                                 "tanh" : 0.5, 
                                                 "sin" : 0.1},
                          #activation_function = {'tanh':0.1, 'relu':0.9},
                          feedback = True,
                          random_state = 210) # was 210
esn_pure_pred.fit(X = input_tr, y = target_tr)
score, prediction = esn_pure_pred.test( X = None, y = target_te)
esn_pure_pred.combined_plot()
final_figure_plot(target_te, None, prediction)
score

### BO optimized 

In [ ]:
# %%time
# #was 0.1 noise = 0.1

# experiment0_hps  = {'connectivity': 0.010497199729654356,
#  'spectral_radius': 1.6224205493927002,
#  'n_nodes': int(100.01666259765625),
#  'regularization': 0.019864175793163488,
#  'leaking_rate': 0.044748689979314804,
#  'bias': 0.8152865171432495}


# esn_pure_pred = RcNetwork(**experiment0_hps, activation_function = {'tanh':0.1, 'relu':0.9},
#                           feedback = True,
#                           random_state = 210) # was 210
# esn_pure_pred.fit(X = input_tr, y = target_tr)
# score, prediction = esn_pure_pred.test( X = input_te, y = target_te)
# esn_pure_pred.combined_plot()
# score

In [ ]:
#these hps are new
experiment0_hps  = {'connectivity': 0.54275345999343795,
 'spectral_radius': 1.5731128454208374,
 'n_nodes': 251,
 'regularization': 0.1880535350594487,
 'leaking_rate': 0.03279460221529007,
 'bias': 0.8625065088272095} 

esn_pure_pred = RcNetwork(**experiment0_hps, 
                          activation_function = {"relu" : 0.33, "tanh" : 0.5, "sin" : 0.1},
                          feedback = True,
                          random_state = 210) # was 210
esn_pure_pred.fit(X = input_tr, y = target_tr)
score, prediction = esn_pure_pred.test( X = input_te, y = target_te)
esn_pure_pred.combined_plot()
final_figure_plot(target_te, None, prediction)
score

### Walkthrough part 2 experiments:

If you have time during revision, go ahead and find better noise hps for the problem above.

In [ ]:
# set up the data
problem, idx = 0, 1
input_tr, input_te, target_tr, target_te = split_data( inputs[problem][idx], datas[problem][idx], 0.2) #
noise = 0.4
#was 0.1

noise_target_tr = target_tr + torch.rand_like(target_tr)*noise
noise_target_te = target_te + torch.rand_like(target_te)*noise

In [ ]:
### with the original experiments:


orig_hps = {'connectivity': 0.4071449746896983,
 'spectral_radius': 1.1329107284545898,
 'n_nodes': 202,
 'regularization': 1.6862021450927922,
 'leaking_rate': 0.009808523580431938,
 'bias': 0.48509588837623596}

In [ ]:
unopt_rc = RcNetwork(**orig_hps, 
                        #activation_function = {"relu" : 0.33, "tanh" : 0.5, "sin" : 0.1},
                        feedback = True,
                        random_state = 210) # was 210
unopt_rc.fit(X = input_tr, y = noise_target_tr)
score, prediction = unopt_rc.test( X = input_te, y = noise_target_te)
score


In [ ]:
unopt_rc.combined_plot(gt_tr_override = target_tr, gt_te_override = target_te)
print(f"score: {score}")
final_figure_plot(target_te, noise_target_te, prediction)

### now with BO optimized hps

In [ ]:

noisep2_hps = {'connectivity': 0.08428374379805789,
 'spectral_radius': 1.1395107507705688,
 'n_nodes': int(251.67526245117188),
 'regularization': 1.3896905679295795,
 'leaking_rate': 0.06542816758155823,
 'bias': 0.24717366695404053}


In [ ]:
rc_noise_p2 = RcNetwork(**noisep2_hps, 
                        #activation_function = {"relu" : 0.33, "tanh" : 0.5, "sin" : 0.1},
                        feedback = True,
                        random_state = 210) # was 210
rc_noise_p2.fit(X = input_tr, y = noise_target_tr)
score, prediction = rc_noise_p2.test( X = input_te, y = noise_target_te)
score


In [ ]:
rc_noise_p2.combined_plot(gt_tr_override = target_tr, gt_te_override = target_te)
print(f"score: {score}")
final_figure_plot(target_te, noise_target_te, prediction)

### Walkthrough part 1 experiments

In [ ]:
problem = 3
idx = 0
input_train, input_test, target_train, target_test = split_data( inputs[problem][idx], datas[problem][idx], 0.2) #0.2
plt.plot(datas[problem][idx][:6000])
plot_data(datas[problem][idx])

In [ ]:
%%time

#main_section
hps = {'connectivity': 0.4071449746896983,
 'spectral_radius': 1.1329107284545898,
 'n_nodes': 202,
 'regularization': 1.6862021450927922,
 'leaking_rate': 0.009808523580431938,
 'bias': 0.48509588837623596}

# opt_hps = {'connectivity': 0.4071449746896983,
#  'spectral_radius': 1.1329107284545898,
#  'n_nodes': round(201.7901153564453),
#  'regularization': 1.6862021450927922,
#  'leaking_rate': 0.009808523580431938,
#  'bias': 0.48509588837623596}

In [ ]:
RANDOM_STATE = 210

In [ ]:
plot_data(datas[problem][idx])

In [ ]:
%%time
esn_pure_pred = RcNetwork(**hps, feedback = True, random_state = RANDOM_STATE) # was 210
esn_pure_pred.fit(y = target_train)
score, prediction = esn_pure_pred.test( y = target_test)
score

In [ ]:
esn_pure_pred.combined_plot()
print(f"score: {score}")
final_figure_plot(target_test, None, prediction)

In [ ]:
%%time
esn_pure_pred = RcNetwork(**hps, 
            random_state = RANDOM_STATE, 
            feedback = True,
            activation_function = "tanh")

esn_pure_pred.fit(X = input_train, 
                  y = target_train)

score, prediction = esn_pure_pred.test(X = input_test,
            y = target_test)



In [ ]:
esn_pure_pred.combined_plot()
final_figure_plot(target_test, None, prediction )
print(f'score : {score}')

In [ ]:
%%time

esn_pure_pred = RcNetwork(**hps, 
                           #reservoir_weight_dist = "normal",
                           #output_activation = "tanh",
                           activation_function = {"tanh" : 0.1, 
                                   "relu" : 0.9, 
                                   "sin": 0.05},
                           random_state = RANDOM_STATE, 
                           feedback = True)
esn_pure_pred.fit(X = input_train, 
                  y = target_train,
                  burn_in = 0)

score, prediction = esn_pure_pred.test(X = input_test,
            y = target_test)
esn_pure_pred.combined_plot()
final_figure_plot(target_test, None, prediction )

In [ ]:
def individual_experiment(observer: bool = False,
                          f_out: bool = False,
                          random_state: int = 210,
                          multi = None
                         ): 
    input_tr = input_train if observer else None
    input_te = input_test if observer else None
    
    out_f = "tanh" if f_out else "identity"
    
    rc = RcNetwork(**opt_hps, 
                           output_activation = out_f,
                           activation_function = multi,
                           random_state = random_state, 
                           feedback = True)
    rc.fit(X = input_tr, 
                  y = target_train,
                  burn_in = 0) 
    score, prediction = rc.test(X = input_te, y = target_test)
    
#     rc.combined_plot()
#     final_figure_plot(target_test, None, prediction )
    print(f'score: {score}')
    return score

In [ ]:
def activation_experiment(random_states, act_f_list, save_path = None):
    results = {"Score" : [], "Observer" : [], "f_out" : [],"random_state" : [], "multi" : []}
    for observer in [True, False]:
        for f_out in [True, False]:
            for random_state in random_states:
                for i, act_f_dict in enumerate(act_f_list):
                    score = individual_experiment(observer, f_out, random_state, act_f_dict)
                    results["Score"].append(float(score))
                    results["Observer"].append(observer)
                    results["f_out"].append(f_out)
                    results["random_state"].append(random_state)
                    results["multi"].append(i)
    results = pd.DataFrame(results)
    #save line: pd.save...
    return results
        

In [ ]:
act_f_list = ["tanh", "relu", "sin", 
              {"tanh" : 0.1, "relu" : 0.9},
              {"relu" : 0.9, "tanh" : 0.1},
              {"tanh" : 0.1, "sin" : 0.9},
              {"sin" : 0.9, "tanh" : 0.1},
              {"sin" : 0.9, "relu" : 0.1},
              {"relu" : 0.9, "sin" : 0.1},
              {"relu" : 0.5, "sin" : 0.5},
              {"relu" : 0.5, "tanh" : 0.5},
              {"tanh" : 0.5, "sin" : 0.5},
              {"tanh" : 0.33, "relu" : 0.33, "sin" : 0.33},
              {"tanh" : 0.1, "relu" : 0.85, "sin" : 0.05},
              {"tanh" : 0.85, "relu" : 0.1, "sin" : 0.05},
              {"sin" : 0.85, "relu" : 0.1, "tanh" : 0.05},
              {"sin" : 0.85, "relu" : 0.05, "tanh" : 0.1},
              {"tanh" : 0.1, "relu" : 0.8, "sin" : 0.1},
              {"tanh" : 0.8, "relu" : 0.1, "sin" : 0.1},
              {"sin" : 0.8, "relu" : 0.1, "tanh" : 0.1}]

In [ ]:
%%time
RUN_ACTIVATION_EXPERIMENTS = False
if RUN_ACTIVATION_EXPERIMENTS:
    df_ = activation_experiment([108, 109, 209, 210], act_f_list)
    df_.head()

In [ ]:
#df_.to_csv("rctorch_experiments2.csv")

## noise 

In [ ]:
datas, inputs = synthetic_data(desired_length=np.pi * 100)

In [ ]:
noise_hps = {'n_nodes': 500,
 'connectivity': 0.6475559084256522,
 'spectral_radius': 1.0265705585479736,
 'regularization': 61.27292863528506,
 'leaking_rate': 0.010949543677270412,
 'bias': 0.5907618999481201,
      }

In [ ]:
problem = 2
idx = 2
input_train, input_test, target_train, target_test = split_data( inputs[problem][idx], datas[problem][idx], 0.4)
plt.plot(datas[problem][idx][:4000])
final_figure_plot(datas[problem][idx], None, datas[problem][idx])

In [ ]:
# # input_train, input_test, target_train, target_test = input_train2, input_test2, target_train2, target_test2
# # plt.plot(datas[problem][idx][:4000])
# final_figure_plot(datas[problem][idx], None, datas[problem][idx])

In [ ]:
noise_sigma =  0.5
noisy_target_tr = target_train + torch.rand_like(target_train) * noise_sigma
noisy_target_te = target_test + torch.rand_like(target_test) * noise_sigma

In [ ]:
%%time
RC_noise1 = RcNetwork(**noise_hps, feedback = True, 
            random_state = RANDOM_STATE) # was 210
RC_noise1.fit(y = noisy_target_tr)
score, prediction = RC_noise1.test( y = noisy_target_te)


In [ ]:
RC_noise1.combined_plot(gt_tr_override = target_train, gt_te_override = target_test)
print(f"score: {score}")
final_figure_plot(target_test, noisy_target_te, prediction)

In [ ]:
%%time
rc_noise2 = RcNetwork(**noise_hps, 
            random_state = RANDOM_STATE, 
            feedback = True,
            activation_function = "tanh")

rc_noise2.fit(X = input_train, 
                  y = noisy_target_tr, 
                  burn_in = 0)

score, prediction = rc_noise2.test(X = input_test,
            y = noisy_target_te)

In [ ]:
rc_noise2.combined_plot(gt_tr_override = target_train, gt_te_override = target_test)
print(f"score: {score}")
final_figure_plot(target_test, noisy_target_te, prediction)

In [ ]:
final_figure_plot(target_test,  None , prediction )

In [ ]:
def inverse_hyperbolic_tangent(z):
#     z_max = z.abs().max() + 0.001
#     z = z/z_max
    return (1/2)*(torch.log(1+z) - torch.log(1-z))

In [ ]:
%%time
hps = {'n_nodes': 500,
 'connectivity': 0.6475559084256522,
 'spectral_radius': 1.0265705585479736,
 'regularization': 61.27292863528506,
 'leaking_rate': 0.010949543677270412,
 'bias': 0.5907618999481201,
      }

# extra_hps = {"input_connectivity" : 0.2,
#             "feedback_connectivity" : 0.2}

esn_pure_pred = RcNetwork(**hps,
            activation_function = ["tanh"],
            random_state = 210, 
            feedback = True)
esn_pure_pred.fit(X = input_train, 
            y = target_train, 
            burn_in = 0)
score, _ = esn_pure_pred.test(X = input_test,
            y = target_test)
esn_pure_pred.combined_plot()
print(f'score: {score}')

In [ ]:
final_figure_plot(target_test,  None , prediction )

### Noise comparison

In [ ]:
class Noise_comp:
    """ A function that does noise comparison. Many noise levels for different stuff.
    
    """
    def __init__(self, noise_start, noise_stop, noise_step):
        self.noise_levels = list(np.arange(noise_start, noise_stop, noise_step))
        self.columns = ["x", "xP", "noise", "t"]
        self.count = 0
        
    def run_experiment(self, observers):
        for noise in self.noise_levels:
            print("noise", noise)
            for i in range(1):
                #get the data
                exp2_splitter = Splitter(force_pend_data, noise = True, std = noise)
                a, b = exp2_splitter.split()

                #fit the esn
                if observers:
                    my_esn = My_esn(**noise_hps_obs, 
                            random_state = 210, 
                            feedback = 1,
                            n_inputs = 1,
                            n_outputs = 2)
                    my_esn.fit(X =  None,
                                y = a, 
                                burn_in = 0, gt_override=target_train)
                    my_esn.test(X = None,
                                gt_override = target_test,
                                y = b)
                else:
                    my_esn = My_esn(**noise_hps, 
                            random_state = 210, 
                            feedback = 1,
                            n_inputs = 1,
                            n_outputs = 2)
                    my_esn.fit(X =  input_train[:,1].view(-1,1),
                                y = a, 
                                burn_in = 0, gt_override=target_train)
                    my_esn.test(X = input_test[:,1].view(-1,1),
                                gt_override = target_test,
                                y = b)
                this_round_tr = my_esn.tr_resids
                this_round_te = my_esn.te_resids

                noise_tr = torch.ones_like(this_round_tr[:, 0].reshape(-1,1)).numpy() * noise
                noise_te = torch.ones_like(this_round_te[:, 0].reshape(-1,1)).numpy() * noise

                noise_tr =  np.round(noise_tr, 4)
                noise_te = np.round(noise_te, 4)

                this_round_tr = this_round_tr.numpy()
                this_round_te = this_round_te.numpy()
                #assert False, f'this_round_tr {this_round_tr.shape}this_round_te {this_round_te.shape}'

                t_tr = np.array(range(len(noise_tr))).reshape(-1,1)/40
                t_te = np.array(range(len(noise_te))).reshape(-1,1)/40

                if self.count == 0:
                    print("building d")
                    Data_tr = pd.DataFrame(this_round_tr)
                    Data_te = pd.DataFrame(this_round_te)
                    Data_tr["noise"] = noise_tr
                    Data_te["noise"] = noise_te

                    Data_tr["t"] = t_tr
                    Data_te["t"] = t_te
                    
                    Data_tr.columns = self.columns
                    Data_te.columns = self.columns

                else:
                    new_data_tr = np.concatenate((this_round_tr, noise_tr, t_tr), axis = 1)
                    new_data_te = np.concatenate((this_round_te, noise_te, t_te), axis = 1)

                    new_data_tr = pd.DataFrame(new_data_tr)
                    new_data_te = pd.DataFrame(new_data_te)

                    new_data_tr.columns = self.columns
                    new_data_te.columns = self.columns

                    Data_tr = pd.concat((Data_tr, new_data_tr), axis = 0)
                    Data_te = pd.concat((Data_te, new_data_te), axis = 0)

                self.count +=1
        self.Data_tr = Data_tr
        self.Data_te = Data_te
        
    def loss_plot(self, var = "x", data = "te"):
        if data == "te":
            df = self.Data_tr
        elif data == "tr":
            df = self.Data_te
        
        fig, ax = plt.subplots(1,1, figsize = (16, 4))
        self.g = sns.lineplot( x = 't', y = var, data = df, ax = ax, hue = "noise")
        plt.yscale('log')
        xlabels = ['{:,.4f}'.format(x) for x in self.g.get_xticks()/10]
        self.g.set_xticklabels(xlabels)
        plt.show()

In [ ]:
NOISE_EXPERIMENTS = False
if NOISE_EXPERIMENTS:
    noise_comp = Noise_comp(0.01, 3.0, 0.05)
    noise_comp.run_experiment(observers = False)
    noise_comp.loss_plot(data = "tr")
    noise_comp.loss_plot(data = "te")
    noise_comp.loss_plot(var = "xP", data = "te")

In [ ]:
if NOISE_EXPERIMENTS:
    noise_comp_obs = Noise_comp(0.01, 3.0, 0.05)
    noise_comp_obs.run_experiment(observers = True)
    noise_comp_obs.loss_plot(data = "tr")
    noise_comp_obs.loss_plot(data = "te")
    noise_comp_obs.loss_plot(var = "xP", data = "te")

In [ ]:
#noise_comp_obs.Data_te.head()
#noise_comp_obs.Data_te["x"]
#print(noise_comp_obs.Data_te["x"].median(), noise_comp.Data_te["x"].median() )
#print(noise_comp_obs.Data_te["xP"].median(), noise_comp.Data_te["xP"].median() )

### Below are noise bar plots

In [ ]:
my_esn.te_resids.reshape(2,-1).shape
plt.plot(my_esn.te_resids.reshape(-1,2))

### This section will be a force comparison
1. Marios's idea
2. evaluated on different forces

We need a class that can easily store the different data that we want. ie. cos for different alpha and phi

$$F \propto \alpha f(\omega T)$$

two plots: {$\omega$ vs. L}, {$\omega$ vs. $\alpha$}
$$f \in {\alpha sin(\omega), cos(\omega), \alpha * sin(\omega)*cos(\omega), sin(2\omega)}$$

In [ ]:
class Fp_DataGen:
    """ A class that generates and stores force_pend data"""
    
    def __init__(self, A_range, W_range, 
                 Nt = 20000, length = 100*np.pi, 
                 x0= .5, px0 = .5, dt = None, split = 0.6, non_resonant_only = True,
                 threshold = 5, force = "sin"):
        """
        Arguments:
            Nt: number of time points
            length: number of datapoints
            dt: upgrade later so we can take dt instead of Nt and length
            x0, px0: initial position and momentum
            A_range: the range of alpha (should be np.arrange)
            W_range: the range of W  (should be np.arrange)
        """
        #original was 8000, and 40 pi
        #new is  20000 and 100 pi to preserve dt
        
        self.x0, self.px0 = x0, px0
        t = np.linspace(0, length, Nt)
        self.datasets = []
        
        for i, a in enumerate(A_range):
            for j, w in enumerate(W_range):
                my_fp = Fpendulum(t = t, x0 = x0, px0 = px0, A = a, W = w, force = force)
                data = my_fp.force_pend_data
                force_ = my_fp.force(A = a, W = w, t = t)
                t = my_fp.t
                
                if np.max(np.abs(data)) > threshold:
                    #resonant.append(1)
                    resonant = 1
                else:
                    resonant = 0
                
                fp_data_spec = {"a": a, "w": w, 
                                "data" : data,
                                "force" : force_,
                                "t" : t,
                                "resonant" : resonant}
                
                #enforce typing:
                for key, val in fp_data_spec.items():
                    if key != "resonant":
                        fp_data_spec[key] = torch.tensor(val, dtype = torch.float32)
                if non_resonant_only:
                    if resonant == 0:
                        self.datasets.append(fp_data_spec)
                else:
                    self.datasets.append(fp_data_spec)
        self.find_resonance()
        
    def plot_all(self):
        for dictt in self.datasets:
            plt.plot(dictt["data"], alpha = 0.1)
    
    def find_resonance(self, threshold = 10):
        resonant = []
        for i, dictt in enumerate(self.datasets):
            if torch.max(torch.abs(dictt["data"])) > threshold:
                resonant.append(1)
                self.datasets[i]["resonant"] = 1
            else:
                self.datasets[i]["resonant"] = 0
                resonant.append(0)
        
        return torch.tensor(resonant, dtype = torch.int32).reshape(-1,1)
    
    @staticmethod
    def _find_resonance(self, datasets, threshold = 10):
        resonant = []
        for i, dictt in enuemerate(datasets):
            
            if torch.max(torch.abs(dictt["data"])) > threshold:
                
                resonant.append(1)
            else:
                
                resonant.append(0)
        
        return torch.tensor(resonant, dtype = torch.int32).reshape(-1,1)
    
    def plot_resonant(self, threshold):
        plt.figure(figsize = (16, 4))
        rez = self.find_resonance(threshold)
        flag = 0
        for i, resonant_bool in enumerate(rez):
            if resonant_bool == 1:
                abs_data = torch.abs(self.datasets[i]['data'])
                abs_px = abs_data[:,0]
                abs_x = abs_data[:,1]
                if flag == 0:
                    plt.plot(abs_x, color = "red", alpha = 0.1, label = "x")
                    plt.plot(abs_px, color = "blue", alpha = 0.1, label = "px")
                    flag = 1
                else:
                    plt.plot(abs_x, color = "red", alpha = 0.1)
                    plt.plot(abs_px, color = "blue", alpha = 0.1)
                    
        plt.axhline(y = threshold, color = "black", label = "threshold")
        plt.yscale('log')
        plt.legend()
        plt.title("resonant")
        plt.show()
        
        for i, resonant_bool in enumerate(rez):
            if resonant_bool == 0:
                data = self.datasets[i]['data']
                abs_px = data[:,0]
                abs_x = data[:,1]
                plt.plot(abs_x, color = "red", alpha = 0.1)
                plt.plot(abs_px, color = "blue", alpha = 0.1)
        plt.legend()
        plt.title("non-resonant")
        plt.show()



In [ ]:
dt = np.pi/200
(dt * 20000)/np.pi

In [ ]:
# #original was 8000, and 40 pi
# #new is  20000 and 100 pi to preserve dt


#my_fp.plot_()



In [ ]:
#small_dataset.find_resonance()
PLOT_RESONANCE = False
if PLOT_RESONANCE:
    small_dataset.plot_resonant(5)

In [ ]:
@ray.remote(max_calls=1)
def evaluate_rcs(datasets, i, hps, observers = True):#, target_tr, target_te, obs_tr, obs_te):
    
    dataset = datasets[i]
    print(f'datasets: {dataset["a"]}')
    
    t, force, data = dataset["t"], dataset["force"], dataset["data"]

    #for now just do the pure pred.
    t_splitter = Splitter(t, noise = False)
    ttrain, ttest = t_splitter.split()

    data_splitter = Splitter(data, noise = False)
    target_train, target_test = data_splitter.split()
    
    force_splitter = Splitter(force, noise = False)
    input_train, input_test = force_splitter.split()
    
    #datasets[0]["force"]
    
    my_esn = RcNetwork(**hps, 
                    random_state = 210, 
                    feedback = 1,
                    activation_function = {"tanh" : 0.1, 
                           "relu" : 0.9, 
                           "sin": 0.05},)
    
    if observers:
        fit = my_esn.fit(X = input_train, 
                    y = target_train,
                    burn_in = 0) #gt_override=target_train)

        val_scores, pred_ = my_esn.test(X = input_test, 
                                        #gt_override=target_test,
                                        y = target_test)
    else:
        fit = my_esn.fit(X =  None, #input_train, 
                    y = target_train,
                    burn_in = 0) #gt_override=target_train)

        val_scores, pred_ = my_esn.test(X = None, #input_test,
                    #gt_override=target_test,
                    y = target_test)

    resids = ( pred_ - target_test) ** 2 #my_esn.te_resids

    #we need x and px eventually, this is fine for now
    max_resid  = torch.max(resids)
    mean_resid = torch.mean(resids)
    
    #tensors2save = {}
    
    data2save = {"observers" : False,
                 "max_resid" : max_resid,
                 "mean_resid" : mean_resid,
                 "a" : dataset["a"],
                 "w" : dataset["w"],
                 "resids" : resids,
                 "tr_target" : target_train.numpy(),
                 "tr_pred" : fit,
                 "te_target" :  target_test.numpy(),
                 "te_pred" : pred_.numpy()}
    return data2save#, tensors2save


In [ ]:
#small_dataset.datasets

In [ ]:
# obs_inputs = {"target_tr" : a,
#               "target_te" : b,
#               "obs_tr" : None,
#               "obs_te" : None}
def preprocess_parallel_batch(lst, batch_size):
    iters = []
    floor_div = len(lst)//batch_size
    remainder = len(lst) % batch_size
    for i in range(floor_div):
        iters.append(batch_size)
    if remainder != 0:
        iters += [remainder]
    start_index = 0
    batched_lst = []
    for iterr in iters:
        stop_index = start_index + iterr
        batched_lst.append(lst[start_index:stop_index])
    return batched_lst

def retrieve_dataset(dataset_obj, hps, batch_size = 9):
    
    #     range_ = range(0, max(len(dataset_obj.datasets) + 1, batch_size + 1), batch_size)
    rez = []
    datasets = dataset_obj.datasets.copy()
    datasets_id = ray.put(datasets)
    batch_indices = preprocess_parallel_batch(list(range(len(datasets))), batch_size)
    total_idx = 0
    
    hps_id = ray.put(hps)
    for i, sub_batch in enumerate(batch_indices):
        start = sub_batch[0] + total_idx
        stop = sub_batch[-1]+ total_idx
        
        print(f'start {start} stop {stop}')
        
        this_set_indexs = list(range(start, stop))
        #time.sleep(0.5)
        
        results = ray.get([evaluate_rcs.remote(datasets_id, i, hps_id) for i in this_set_indexs])
        
        rez+=results
        #         if i == 2:
        #             breakpoint()
        #evaluate_rcs_plain(datasets_spec[0], hps)
        total_idx += (stop - start)
        print(f'percent_complete {i/len(batch_indices) * 100}')
    return rez

In [ ]:
def preprocess_parallel_batch(lst, batch_size):
    iters = []
    floor_div = len(lst)//batch_size
    remainder = len(lst) % batch_size
    for i in range(floor_div):
        iters.append(batch_size)
    if remainder != 0:
        iters += [remainder]
    start_index = 0
    batched_lst = []
    for iterr in iters:
        stop_index = start_index + iterr
        batched_lst.append(lst[start_index:stop_index])
    return batched_lst

In [ ]:
%%time
# noise_hps = {'n_nodes': 500,
#  'connectivity': 0.20541412114258625,
#  'spectral_radius': 1.4592119455337524,
#  'regularization': 7.651269704734278,
#  'leaking_rate': 0.017093051224946976,
#  'bias': 0.36459600925445557}
noise_hps = {'connectivity': 0.4071449746896983,
 'spectral_radius': 1.1329107284545898,
 'n_nodes': 202,
 'regularization': 1.6862021450927922,
 'leaking_rate': 0.009808523580431938,
 'bias': 0.48509588837623596}

In [ ]:
def make_results_dfs(rez_, fp = None):
    
    
    full_data_keys = ["tr_target", "tr_pred", "te_target", "te_pred", "resids"]
    pd_keys = ["observers", "max_resid", "mean_resid", "a", "w"]
    
    #pd_results = pd.DataFrame(rez_[pd_keys])
    data_summaries = []
    
    for dict_ in rez:
        data_summaries.append({key: float(val) for key, val in dict_.items() if key in pd_keys})
    
#     for results in rez:
#         try :
#             pd_results[col] = pd_results[col].astype(float)
#         except:
#             pass #assert False, f'{col} {pd_results[col]}'
#     pd_results.head()
    if fp:
        pd_results.to_csv(fp)
    return pd.DataFrame(data_summaries), rez_

In [ ]:
#a_range, w_range

In [ ]:
import pickle

a_range = np.arange(0.1, 1.0, 0.05)
w_range = np.arange(0.1, 1.0, 0.05)

force_experiments = False
if force_experiments:
    for force__ in ["sin", "cos", "sincos"]:
        small_dataset = Fp_DataGen(A_range = a_range, W_range = w_range, Nt = 8000, length = 40*np.pi,
                                   force = force__)
        rez = retrieve_dataset(small_dataset, noise_hps)
        pd_results, all_data = make_results_dfs(rez)
        print(pd_results)
        fp_base = './new_results/' + force__
        fp1 = fp_base +'_results.csv'
        fp2 = fp_base + '_all_data.pickle'
        pd_results.to_csv(fp1)
        with open(fp2, 'wb') as handle:
            pickle.dump(all_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

#         

# print a == b

In [ ]:
force = "cos"
val = "mean_resid"

fp = "new_results/" + force + "_results.csv"
fp2 = "new_results/"+ force + "_all_data.pickle"

pd_results_ = pd.read_csv(fp)

In [ ]:
#print(pd_results_["max_resid"][0])
from matplotlib.colors import LogNorm, Normalize

with open(fp2, 'rb') as handle:
    cos_results = pickle.load(handle)
#cos_results

plt.title(force + " force: " + val)
pivot = pd_results_.pivot(index='a', columns='w', values=val)
g = sns.heatmap(pivot, vmax = 1, vmin = 0, norm=LogNorm(), cmap = "cubehelix")  #)#np.log10(pivot))
g.set_facecolor('lightpink')
yticklabels = g.get_yticklabels()
xticklabels = g.get_xticklabels()
g.set_yticklabels([round(float(yticklabels[i].get_text()),3) for i in  range(len(yticklabels))])
g.set_xticklabels([round(float(xticklabels[i].get_text()),3) for i in  range(len(xticklabels))])

#ylabels = ['{:,.2f}'.format(x) for x in g.get_yticks()]
#g.set_yticklabels(ylabels)
plt.show()

In [ ]:
g.set_yticklabels() 

In [ ]:
print(cos_results[0].keys())

def plot_pred_(i, plot_test_set = True, phase_plot = True, trajectories = False):
    if plot_test_set:
        keys = "te_target", "te_pred"
    else:
        keys = "tr_target", "fit"
    
    #plt.figure(figsize = (6,2))
    cos_result = cos_results[i]
    print(cos_result.keys())
    
    pred = cos_result[keys[1]]
    gt = cos_result[keys[0]]
    
    
    if trajectories:
        fig, ax = plt.subplots(1,2, figsize = (14,3))
        ax[0].plot(pred, '--', linewidth = 2)
        ax[0].plot(gt, alpha = 0.5, linewidth = 2)

        ax[1].plot((cos_results[i][keys[0]] - cos_results[i][keys[1]])**2, '--', linewidth = 2)
        #ax[0].plot(cos_results[i]["fit"], alpha = 0.5, linewidth = 2)

        ax[1].set_yscale("log")
    if phase_plot:
        final_figure_plot(pred, None, gt)
    plt.show()
for i in range(len(cos_results)):
    plot_pred_(i)

## Run Experiments

In [ ]:
%%time

def run_experiments(datas, inputs, hps, split, output_activation,  noise = None, activation_dict = "tanh"):
    #split = 0.6
    scores = []
    for i, data in enumerate(datas):
        for j, trajectory in enumerate(data):
            force_pend_data__ = trajectory
            input_tr, input_te, target_tr, target_te = split_data( inputs[i][j], trajectory, split)
           


            input_train, input_test, target_train, target_test = split_data( input_, force_pend_data, split)
            #input_train, input_test, target_train, target_test = split_data( input_, force_pend_data, 0.6)
            

            esn_pure_pred = RcNetwork(**hps, 
                        random_state = 210, 
                        feedback = True,
                        output_activation = output_activation,
                        activation_function = activation_dict)
                        #solve_sample_prop = 0.8
                                            
            if noise is not None:
                exp2_splitter = Splitter(trajectory, noise = True, std = noise, split = split)
                noisy_tr_target, noisy_te_target = exp2_splitter.split()
                
                esn_pure_pred.fit(X = noisy_tr_target, #input_tr,#None, 
                            y = target_tr, 
                            burn_in = 0)
                score, prediction = esn_pure_pred.test(X = noisy_te_target,
                            y = target_te)
                final_figure_plot(test_gt = target_te, noisy_test_gt = noisy_te_target, rc_pred = prediction, 
                      noisy_format = '.')
            else:
                esn_pure_pred.fit(X = input_tr, #input_tr,#None, 
                            y = target_tr, 
                            burn_in = 0)
                score, prediction = esn_pure_pred.test(X = input_te,
                            y = target_te)
                final_figure_plot(test_gt = target_te, noisy_test_gt = None, rc_pred = prediction, 
                      noisy_format = '.')
                
#             esn_pure_pred.combined_plot(gt_tr_override=target_tr,
#                                         gt_te_override=target_te)
            
            print(f'score : {score}')
            scores.append(score)
            
            plt.show()
    total_score = np.mean(scores)
    print(f'total score {total_score}')

opt_hps = {'connectivity': 0.4071449746896983,
 'spectral_radius': 1.1329107284545898,# 1.1329107284545898,
 'n_nodes': round(201.7901153564453),
 'regularization': 1.6862021450927922,
 'leaking_rate': 0.009808523580431938,
 'bias': 0.48509588837623596,
 'input_connectivity' : 0.3}

run_experiments(datas, inputs, hps = opt_hps, split = 0.6, output_activation = "identity")

run_experiments(datas, inputs, hps = opt_hps, split = 0.5, output_activation = "tanh", noise = 0.04)

In [ ]:
experiment0_hps  = {'connectivity': 0.010497199729654356,
 'spectral_radius': 1.6224205493927002,
 'n_nodes': int(100.01666259765625),
 'regularization': 0.019864175793163488,
 'leaking_rate': 0.044748689979314804,
 'bias': 0.8152865171432495}
run_experiments(datas, inputs, hps = opt_hps, split = 0.5,
                activation_dict = {"relu" : 0.33, "tanh" : 0.5, "sin" : 0.1},
                output_activation = "tanh", noise = 0.04)

In [ ]:
# pd_results.to_csv('./fp_sin_results.csv')
# pd_results.to_csv('./fp_sin_pd_max_resid.csv')
# #pd_results.to_csv('./fp_sin_pd_mean_resid.csv')


In [ ]:
#TODO add this function to esn_cv
def recover_hps(self):
    best_vals = self.X_turbo[torch.argmax(self.Y_turbo)]
        
    denormed_ = self.denormalize_bounds(best_vals)

    try:
        denormed_ = denormalize_bounds(best_vals)
    except:
        print("FAIL")

    #best_vals = X_turbo[torch.argmax(Y_turbo)]

    #####Bad temporary code to change it back into a dictionaryf
    denormed_free_parameters = list(zip(self.free_parameters, denormed_))
    denormed_free_parameters = dict([ (item[0], item[1].item()) for item in denormed_free_parameters])

    best_hyper_parameters = denormed_free_parameters
    for fixed_parameter in self.fixed_parameters:
        best_hyper_parameters = {fixed_parameter : self.bounds[fixed_parameter], **best_hyper_parameters }

    #log_vars = ['connectivity', 'llambda', 'llambda2', 'noise', 'regularization', 'dt']
    for var in self.log_vars:
        if var in best_hyper_parameters:
            best_hyper_parameters[var] = 10. ** best_hyper_parameters[var] 



    # Return best parameters
    return best_hyper_parameters

## below this point is Hennon Hailes

###  Data: <a href='https://en.wikipedia.org/wiki/Hénon–Heiles_system'>The Henon-Heiles System</a>

In [ ]:
def residuals(prediction,target):
    try:
        prediction = prediction.numpy()
    except:
        pass
    try:
        target = target.numpy()
    except:
        pass
    return (target.flatten() - prediction.flatten())

In [ ]:
#helper functions:
def plot_data(train, test, ydata):
    plt.figure(figsize=(14,8))
    plt.subplot(2,1,1)
    if isinstance(train, np.ndarray):
        train, test = torch.tensor(train), torch.tensor(test)
    trainlen, testlen = len(train), len(test)
    plt.plot(range(0,trainlen+testlen),hstack((train,test)),'k',label='data')
    plt.plot(range(0,trainlen), train,'g',label='train')
    plt.plot(range(trainlen,trainlen+testlen), test,'-r',label='test')
    plt.ylabel('x(t)')
    plt.legend(loc=(0.1,1.1),fontsize=18,ncol=3)
    plt.tick_params(labelbottom=False)
    plt.subplot(2,1,2)
    plt.plot(tdata,ydata,'k')
    plt.ylabel('y(t)')
    plt.tight_layout()
    plt.show()

def plotResults(trainlen, testlen, data, yfit, yhat, resTrain, resTest):
    """
    Arguments:
        trainlen: the length of the training set
        testlen: the length of the test set
        data: the entire dataset
        yfit: the prediction of the RC on the training set 
        yhat: the prediction of the RC on an unseen test set
        resTrain: training residuals
        resTest: test residuals
    """
    #data plot
    plt.figure(figsize=(14,6))
    plt.subplot(2,1,1)
    plt.plot(range(0,trainlen+testlen),data[:(trainlen+testlen)],'k',  linewidth=2, label="data")
    plt.plot(range(0,trainlen),yfit,'--g',  linewidth=2, alpha=0.9, label="train")
    plt.plot(range(trainlen,trainlen+testlen), yhat,'--r', linewidth=2,  alpha=1, label="test")
    lo,hi = plt.ylim()
    plt.plot([trainlen,trainlen],[lo+np.spacing(1),hi-np.spacing(1)],'--b',alpha=0.8, linewidth=4)
    plt.tight_layout()
    plt.ylabel('x')
    plt.legend(loc=(0.1,1.1),fontsize=18,ncol=3)

    
    #Residuals plot
    plt.subplot(2,1,2)
    plt.plot(range(0,trainlen),  np.abs(resTrain),'--g',  linewidth=2, alpha=0.9, label="train")
    plt.plot(range(trainlen,trainlen+testlen), np.abs(resTest),'--r', linewidth=2,  alpha=1, label="test")
    lo, hi = plt.ylim()
    plt.yscale("log10") 
    plt.plot([trainlen,trainlen],[lo+np.spacing(1),hi-np.spacing(1)],'--b',alpha=0.8, linewidth=4)
    plt.tight_layout()
    plt.ylabel('Residuals')



### Load the data:

In [ ]:
# lines = loadtxt("filename.dat", comments="#", delimiter=",", unpack=False)
tdata = loadtxt("data/HenonHeiles/t.dat")
xdata = loadtxt("data/HenonHeiles/x.dat")
ydata = loadtxt("data/HenonHeiles/y.dat")

tdata = tdata.astype('float32')
xdata = xdata.astype('float32')
ydata = ydata.astype('float32')

In [ ]:





# N = len(tdata)
# print("The time series consist of ", N, " points.")

# plt.figure(figsize=(14,6))
# plt.subplot(2,1,1)
# plt.plot(tdata,xdata,'k')
# plt.xlabel('t')
# plt.ylabel('x(t)')
# plt.subplot(2,1,2)
# plt.plot(tdata,ydata,'k')
# plt.ylabel('y(t)')
# plt.tight_layout()
# plt.show()

#split the forecast:
split = 0.45
trainlen = int(split*N)
testlen  = int((1-split)*N)
# trainlen = int(0.5*N)
# testlen  = int(0.5*N)


# Input  data equivalents:
#ttrain = np.ones(trainlen)
#ttest=np.ones(testlen)

# ttrain = tdata[:trainlen]
# ttest  = tdata[trainlen:trainlen+testlen]

ytrain = ydata[:trainlen]
ytest  = ydata[trainlen:trainlen+testlen]

# Output data
xtrain = xdata[:trainlen]
xtest = xdata[trainlen:trainlen+testlen]

# plt.figure(figsize=(14,2))

# plt.plot(range(0,trainlen), xtrain,'r')
# plt.plot(range(trainlen,trainlen+testlen), xtest,'-g')
# # plt.plot(ttrain, xtrain,'b')
# # plt.plot(ttest, xtest,'-r')
plot_data(xtrain, xtest, ydata)

In [ ]:
my_splitter = Splitter(xdata, noise = True)
xtrain_noisy, xtest_noisy = my_splitter.split()

plot_data(xtrain_noisy, xtest_noisy, ydata)

In [ ]:
pure_pred_noise_hps = {'n_nodes': 500,
 'connectivity': 0.12408451143662941,
 'spectral_radius': 1.2, #1.1268014907836914,
 'regularization': 0.022762411251526036,
 'leaking_rate': 0.2863200306892395,
 'bias': 0.32563644647598267}

pp_noise_esn = My_esn(**pure_pred_noise_hps, 
            random_state = 210, 
            feedback = 1,
            n_inputs = 1,
            n_outputs = 2)
pp_noise_esn.fit(X = None, 
            y =  a,
            burn_in = 0,
            gt_override = target_train)
pp_noise_esn.test(X = None,
            y =  b,
            gt_override = target_test)

In [ ]:
pp_noise_esn.combined_plot()

In [ ]:
#declare the bounds dict. See above for which variables are optimized in linear vs logarithmic space.
bounds_dict = {"connectivity" : (-1, -0.1), 
               "spectral_radius" : (1, 2),
               "n_nodes" : 500,
               "regularization" : (-3, 3),
               "leaking_rate" : (0, 1),
               #"input_scaling" : (0, 1),
               #"feedback_scaling" : (0, 1),
               "bias": (0,1)
               }

#declare the esn_cv optimizer: this class will run bayesian optimization to optimize the bounds dict.

esn_cv = EchoStateNetworkCV(bounds = bounds_dict, esn_feedback = True, 
                            scoring_method = "nmse", interactive = True, 
                            n_jobs = 4, cv_samples = 1, initial_samples = 8, 
                            subsequence_length = int(ytrain.shape[0] * 0.8),
                            random_seed = 209, success_tolerance = 3, ODE_order = None,
                            length_min = 2**-11,
                            validate_fraction = 0.5,
                            n_inputs = 1,
                            n_outputs = 1
                            
           )
#optimize:
opt_hps = esn_cv.optimize( n_trust_regions = 2, max_evals = 500,
                            x = xtrain_noisy.reshape(-1,1), y =ytrain.reshape(-1,1))

In [ ]:
opt_hps

In [ ]:
# first_denoise_hps = {'n_nodes': 200,
#  'connectivity': 0.29002750025150953,
#  'spectral_radius': 1.2885233163833618,
#  'regularization': 0.013653433082063734,
#  'leaking_rate': 0.5670961141586304,
#  'bias': 0.3260454535484314}

noise_hps = {'n_nodes': 500,
 'connectivity': 0.32094487651215875,
 'spectral_radius': 1.23499596118927,
 'regularization': 0.8368901513227046,
 'leaking_rate': 0.32397231459617615,
 'bias': 0.24689573049545288}
#trained from x to y, appear to be generally good denoising parameters for this equation.

esn = EchoStateNetwork(**noise_hps, random_state = 209, feedback = 1, n_inputs = 1, n_outputs = 1)
yfit = esn.fit(X = xtrain_noisy.reshape(-1,1), y = ytrain, burn_in = 0)
scoreTest, yhat = esn.test(X = xtest_noisy.reshape(-1,1), y =ytest.reshape(-1,1))

# MSE in the training and testing
scoreTrain = myMSE(yfit,ytrain)
scoreTest = myMSE(yhat,ytest)
print("Training mean square error: ",scoreTrain)
print("Testing  mean square error: ",scoreTest)

# Residuals 
resTrain = residuals(yfit, ytrain);
resTest = residuals(yhat, ytest);

# Plot:
plotResults(trainlen, testlen, ydata, yfit, yhat, resTrain, resTest)

In [ ]:
esn = EchoStateNetwork(**noise_hps, random_state = 209, feedback = 1, n_inputs = 1, n_outputs = 1)
xfit = esn.fit(X = xtrain_noisy.reshape(-1,1), y = xtrain, burn_in = 0)
scoreTest, xhat = esn.test(X = xtest_noisy.reshape(-1,1), y =xtest.reshape(-1,1))


# MSE in the training and testing
scoreTrain = myMSE(xfit,xtrain)
scoreTest = myMSE(xhat,xtest)
print("Training mean square error: ",scoreTrain)
print("Testing  mean square error: ",scoreTest)

# Residuals 
resTrain = residuals(xfit, xtrain);
resTest = residuals(xhat, xtest);

# Plot:
plotResults(trainlen, testlen, xdata, xfit, xhat, resTrain, resTest)

In [ ]:
esn.extended_states.shape, esn.X.shape
# optimized_hyper_params = {'n_nodes': 200,
#  'connectivity': 0.4483155375901789,
#  'spectral_radius': 1.5525517463684082,
#  'regularization': 0.005069355134121151,
#  'leaking_rate': 0.6272786855697632,
#  'bias': 0.750773549079895}

In [ ]:
fig, ax = plt.subplots(2,1, figsize = (10,5))
plt.sca(ax[0])
plt.plot(torch.abs(esn.extended_states[:,0:100]), alpha = 0.3)
plt.sca(ax[1])
plt.plot(esn.val_states[:,0:5])

In [ ]:
states2plot = 10
index2start = 10
valstates2plot =1000
combined_states = torch.cat((esn.extended_states[:,1:states2plot+1], esn.val_states[0:valstates2plot, 0:states2plot]))
plt.plot(combined_states[index2start:])
plt.axvline(500-index2start)

In [ ]:
optimized_hyper_params

#### Long range forecast (inference)

In [ ]:
trainlen = int(0.5*N)
testlen  = int(0.5*N)

# Input  data
ttrain = np.ones(trainlen)
ttest=np.ones(testlen)

ytrain = ydata[:trainlen]
ytest  = ydata[trainlen:trainlen+testlen]

# Output data
xtrain = xdata[:trainlen]
xtest = xdata[trainlen:trainlen+testlen]


plt.figure(figsize=(14,2))
plt.plot(range(0,trainlen), xtrain,'r')
plt.plot(range(trainlen,trainlen+testlen), xtest,'-g')
# plt.plot(ttrain, xtrain,'b')
# plt.plot(ttest, xtest,'-r')

## RcTorch Observers Solution

In [ ]:
bounds_dict = {"connectivity" : (-1, -0.1), 
               "spectral_radius" : (1, 2),
               "n_nodes" : 200,
               "regularization" : (-3, 3),
               "leaking_rate" : (0, 1),
               #"input_scaling" : (0, 1),
               #"feedback_scaling" : (0, 1),
               "bias": (0,1)
               }

esn_cv = EchoStateNetworkCV(bounds = bounds_dict, esn_feedback = True, 
                            scoring_method = "nmse", interactive = True, 
                            n_jobs = 1, cv_samples = 2, initial_samples = 10, 
                            subsequence_length = int(ytrain.shape[0] * 0.9),
                            length_min = 2**-11,
                            random_seed = 123)
optimized_hyper_params = esn_cv.optimize(y = xtrain.reshape(-1,1), x = ytrain.reshape(-1,1), 
                                         n_outputs = 1, n_trust_regions = 2, max_evals = 500)

In [ ]:
esn_cv.parallel_arguments.keys()
esn_cv.parallel_arguments['declaration_args']

In [ ]:

opt_hps_ = hyper_params = {'n_nodes': 200,
 'connectivity': 0.13505189245513274,
 'spectral_radius': 1.6877728700637817,
 'regularization': 0.04002294006918671,
 'leaking_rate': 0.2642587423324585,
 'bias': 0.7290011644363403}


In [ ]:
esn = EchoStateNetwork(**opt_hps_, random_state = 123, feedback = 0)
xfit = esn.fit(y = xtrain, X = ytrain, burn_in = 0)
xfit = xfit.reshape(-1)
#scoreTrain, yfit = esn.test(y = ytrain)
#scoreTest, xhat = esn.test(y = xtest, X = ytest)
scoreTest, test_outputs, test_input = esn.test(y = xtest.reshape(-1,1), X = ytest)
xhat = test_outputs["yhat"]


# MSE in the training and testing
scoreTrain = myMSE(xfit, xtrain)
scoreTest = myMSE(xhat, xtest)
print("Training mean square error: ",scoreTrain)
print("Testing  mean square error: ",scoreTest)

# Residuals 
resTrain = residuals(xfit,xtrain);
resTest = residuals(xhat,xtest);

# Plot:
plotResults(trainlen, testlen, xdata, xfit, xhat, resTrain, resTest)

In [ ]:
plt.plot(test_outputs["yhat"])
plt.plot(test_outputs['ytest'])

In [ ]:
plt.plot(esn.X_val_extended.detach())

In [ ]:
plt.plot(esn.extended_states[:,:10]); torch.std(esn.extended_states[:,0])

In [ ]:
esn = EchoStateNetwork(**hyper_params, random_state = 209, feedback = 0)
xfit = esn.fit(y = xtrain, burn_in = 0)
print(xfit[0].shape)
xfit = xfit.reshape(-1)
#return score, {"yhat": y_predicted.data, "ytest": y}, X[self.burn_in:]
scoreTest, test_outputs, test_input = esn.test(y = xtest.reshape(-1,1))
xhat = test_outputs["yhat"]


# MSE in the training and testing
scoreTrain = myMSE(xfit,xtrain)
scoreTest = myMSE(xhat,xtest)
print("Training mean square error: ",scoreTrain)
print("Testing  mean square error: ",scoreTest)

# Residuals 
resTrain = residuals(xfit, xtrain);
resTest = residuals(xhat, xtest);

# Plot:
plotResults(trainlen, testlen, xdata, xfit, xhat, resTrain, resTest)